In [1]:
import numpy as np
from tqdm import tqdm
from scipy.stats import pearsonr, spearmanr
import scipy.sparse as sparse
from scipy.stats import bernoulli, poisson
import analysis_utils_mine as utils

import json
import pandas as pd
import ast
from datetime import datetime
import torch
import pandas as pd
from datetime import datetime, timedelta
import pickle

import matplotlib.pyplot as plt
import xlsxwriter

In [11]:
def get_dataframe_from_annotated_xlsx_file_path(path):
    df = pd.read_excel(path,
                       sheet_name=None,
                       engine='openpyxl')
    df = df['Sheet1']
    df = df[~pd.isnull(df['Topic Name'])]
    return df

In [12]:
annotator1_speeches = get_dataframe_from_annotated_xlsx_file_path('venue_diff_polsci/pre_tbip_annotation_results/annotator_1/topics_for_annotation.xlsx_frazier_speech.xlsx')
annotator1_speeches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 1569
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Topic           50 non-null     object 
 1   Unnamed: 1      0 non-null      float64
 2   Coherence       50 non-null     float64
 3   Polarization    48 non-null     float64
 4   Topic Name      50 non-null     object 
 5   Description     48 non-null     object 
 6   Notes/Comments  10 non-null     object 
dtypes: float64(3), object(4)
memory usage: 3.1+ KB


/workspace/.conda/envs/tbip/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/workspace/.conda/envs/tbip/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [13]:
annotator1_tweets = get_dataframe_from_annotated_xlsx_file_path('venue_diff_polsci/pre_tbip_annotation_results/annotator_1/topics_for_annotation.xlsx_Frazier_tweet.xlsx')
annotator1_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 1569
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Topic           50 non-null     object 
 1   Unnamed: 1      0 non-null      float64
 2   Coherence       50 non-null     float64
 3   Polarization    50 non-null     float64
 4   Topic Name      50 non-null     object 
 5   Description     50 non-null     object 
 6   Notes/Comments  50 non-null     object 
dtypes: float64(3), object(4)
memory usage: 3.1+ KB


/workspace/.conda/envs/tbip/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/workspace/.conda/envs/tbip/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [14]:
annotator2_speeches = get_dataframe_from_annotated_xlsx_file_path('venue_diff_polsci/pre_tbip_annotation_results/annotator_2/topics_for_annotation_Hightower_speech.xlsx')
annotator2_speeches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 1569
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Topic           50 non-null     object 
 1   Unnamed: 1      0 non-null      float64
 2   Coherence       50 non-null     float64
 3   Polarization    41 non-null     float64
 4   Topic Name      50 non-null     object 
 5   Description     43 non-null     object 
 6   Notes/Comments  16 non-null     object 
dtypes: float64(3), object(4)
memory usage: 3.1+ KB


/workspace/.conda/envs/tbip/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/workspace/.conda/envs/tbip/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [15]:
annotator2_tweets = get_dataframe_from_annotated_xlsx_file_path('venue_diff_polsci/pre_tbip_annotation_results/annotator_2/topics_for_annotation_Hightower_tweets.xlsx')
annotator2_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 1569
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Topic           50 non-null     object 
 1   Unnamed: 1      0 non-null      float64
 2   Coherence       50 non-null     float64
 3   Polarization    36 non-null     float64
 4   Topic Name      50 non-null     object 
 5   Description     38 non-null     object 
 6   Notes/Comments  22 non-null     object 
dtypes: float64(3), object(4)
memory usage: 3.1+ KB


/workspace/.conda/envs/tbip/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/workspace/.conda/envs/tbip/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [30]:
np.isnan(dict(zip(annotator2_tweets['Topic'], annotator2_tweets['Notes/Comments']))['Topic 3'])

True

In [21]:
discard_labels_to_one_discard_label_map = {'DISCARD': 'DISCARD',
                                           'Discard': 'DISCARD',
                                           'Disgard': 'DISCARD',
                                           'discard': 'DISCARD'}
                                           

In [45]:
def two_dataframes_to_one_excel_file_for_consensus_labeling(df1, 
                                                            df2, 
                                                            discard_labels_to_one_discard_label_map,
                                                            outpath):
    N = len(df1)
    assert len(df1) == len(df2)
    assert list(df1['Topic']) == list(df2['Topic'])
    
    topics = list(df1['Topic'])
    
    topic_to_name1 = dict(zip(df1['Topic'], df1['Topic Name']))
    topic_to_desc1 = dict(zip(df1['Topic'], df1['Description']))
    topic_to_notes1 = dict(zip(df1['Topic'], df1['Notes/Comments']))
    
    topic_to_name2 = dict(zip(df2['Topic'], df2['Topic Name']))
    topic_to_desc2 = dict(zip(df2['Topic'], df2['Description']))
    topic_to_notes2 = dict(zip(df2['Topic'], df2['Notes/Comments']))
    
    
    out_topics, out_names1, out_descs1, out_notes1, out_names2, out_descs2, out_notes2 = [], [], [], [], [], [], []
    for topic in topics:
        name1 = topic_to_name1[topic]
        name2 = topic_to_name2[topic]
        if name1 in discard_labels_to_one_discard_label_map and name2 in discard_labels_to_one_discard_label_map:
            continue
        else:
            if name1 in discard_labels_to_one_discard_label_map:
                name1 = discard_labels_to_one_discard_label_map[name1]
            if name2 in discard_labels_to_one_discard_label_map:
                name2 = discard_labels_to_one_discard_label_map[name2]
            out_topics.append(topic)
            out_names1.append(name1) 
            out_descs1.append(topic_to_desc1[topic])
            out_notes1.append(topic_to_notes1[topic])
            out_names2.append(name2) 
            out_descs2.append(topic_to_desc2[topic])
            out_notes2.append(topic_to_notes2[topic])
    
    print('Number of Topics for consensus labeling, after discarding those labeled as DISCARD by both annotators = ' + str(len(out_topics)))
    
    workbook = xlsxwriter.Workbook(outpath)
    workbook.formats[0].set_font_size(12)
    worksheet = workbook.add_worksheet()
    worksheet.freeze_panes(1, 0)
    
    # Add a format for the header cells.
    header_format = workbook.add_format({
        'bottom': 1,
        'top':1,
        'font_size':12,
        #'bg_color': '#C6EFCE',
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'align': 'center',
        #'indent': 1,
    })
    
    header_format_with_rborder = workbook.add_format({
        'bottom': 1,
        'top':1,
        'right':1,
        'font_size':12,
        #'bg_color': '#C6EFCE',
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'align': 'center',
        #'indent': 1,
    })

    # Set up layout of the worksheet.
    worksheet.set_column('A:A', 10)
    worksheet.set_column('B:B', 30)
    worksheet.set_column('C:C', 75)
    worksheet.set_column('D:D', 75)
    worksheet.set_column('E:E', 30)
    worksheet.set_column('F:F', 75)
    worksheet.set_column('G:G', 75)
    worksheet.set_column('H:H', 3)
    worksheet.set_column('I:I', 30)
    worksheet.set_column('J:J', 150)
    worksheet.set_row(0, len(out_topics) + 1)
    

    # Write the header cells and some data that will be used in the examples.
    worksheet.write('A1', 
                    'Topic', 
                    header_format)
    worksheet.write('B1', 
                    'Name 1', 
                    header_format)
    
    worksheet.write('C1', 
                    'Description 1', 
                    header_format)
    
    worksheet.write('D1', 
                    'Notes 1',
                   header_format_with_rborder)
    
    worksheet.write('E1', 
                    'Name 2', 
                    header_format)
    worksheet.write('F1', 
                    'Description 2', 
                    header_format)
    
    worksheet.write('G1', 
                    'Notes 2', 
                    header_format_with_rborder)
    
    worksheet.write('H1', 
                    '', 
                    header_format_with_rborder)
    
    worksheet.write('I1', 
                    'Consensus Topic Name', 
                    header_format)
    
    worksheet.write('J1', 
                    'Notes/Comments', 
                    header_format)
    
    rborder = workbook.add_format({'right': 1})
    twrap = workbook.add_format({'text_wrap': True})
    rborder_twrap = workbook.add_format({'right': 1, 'text_wrap': True})
    
    on_row = 2
    for t, n1, d1, notes1, n2, d2, notes2 in zip(out_topics, out_names1, out_descs1, out_notes1, out_names2, out_descs2, out_notes2):
        worksheet.write('A' + str(on_row), t, twrap)
        worksheet.write('B' + str(on_row), n1, twrap)
        if type(d1) == str:
            worksheet.write('C' + str(on_row), d1, twrap)
        worksheet.write('D' + str(on_row), '', rborder)
        if type(notes1) == str:
            worksheet.write('D' + str(on_row), notes1, rborder_twrap)
        worksheet.write('E' + str(on_row), n2, twrap)
        if type(d2) == str:
            worksheet.write('F' + str(on_row), d2, twrap)
        worksheet.write('G' + str(on_row), '', rborder)
        if type(notes2) == str:
            worksheet.write('G' + str(on_row), notes2, rborder_twrap)
        
        worksheet.write('H' + str(on_row), '', rborder)
        on_row += 1
    workbook.close()

In [46]:
two_dataframes_to_one_excel_file_for_consensus_labeling(annotator1_speeches,
                                                        annotator2_speeches,
                                                        discard_labels_to_one_discard_label_map,
                                                        'venue_diff_polsci/consensus_topic_labeling_files/speeches_consensus_labeling.xlsx')

Number of Topics for consensus labeling, after discarding those labeled as DISCARD by both annotators = 49


In [47]:
two_dataframes_to_one_excel_file_for_consensus_labeling(annotator1_tweets,
                                                        annotator2_tweets,
                                                        discard_labels_to_one_discard_label_map,
                                                        'venue_diff_polsci/consensus_topic_labeling_files/tweets_consensus_labeling.xlsx')

Number of Topics for consensus labeling, after discarding those labeled as DISCARD by both annotators = 50
